# Graph-Rewriting Automata
---

### Import required packages

In [1]:
import numpy as np
import tensorflow as tf

2022-09-16 13:27:19.352087: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Class definition

In [363]:
class Graph:
    """
    The Graph class can be instantiated with an adjacency matrix and a state vector.
    """

    def __init__(self, adjacency_matrix, state_vector):
        self.adjacency_matrix = tf.sparse.from_dense(adjacency_matrix)
        self.state_vector = tf.constant(state_vector, dtype=tf.int32)

    def evolve(self, rule):
        # computes the configuration vector
        configurations = tf.sparse.sparse_dense_matmul(self.adjacency_matrix, self.state_vector).numpy().transpose().squeeze() + 4*self.state_vector.numpy().transpose().squeeze()

        # creates a list with the 16 first binary digits of the rule number
        rule = [int(d) for d in np.binary_repr(rule)]
        rule.reverse()
        for i in range(len(rule), 16): rule.append(0)

        # computes an updated state vector and a division vector
        new_state_vector = [rule[c] for c in configurations]
        division_vector =  [rule[c+8] for c in configurations]

        # self.state_vector = new_state_vector

        while 1 in division_vector:
            i = division_vector.index(1)
            dim = len(division_vector)

            # updates the state vector
            new_state_vector.insert(i, new_state_vector[i])
            new_state_vector.insert(i, new_state_vector[i])

            # updates the division vector
            division_vector[i]=0
            division_vector.insert(i, 0)
            division_vector.insert(i, 0)

            # updates the adjacency matrix
            self.adjacency_matrix = tf.sparse.concat(0,
                [
                    tf.sparse.slice(self.adjacency_matrix, [0,0], [i,dim]),
                    tf.SparseTensor(indices=[(tf.sparse.slice(self.adjacency_matrix, [i,0], [1,dim]).indices)[0]], values=[1], dense_shape=[1,dim]),
                    tf.SparseTensor(indices=[(tf.sparse.slice(self.adjacency_matrix, [i,0], [1,dim]).indices)[1]], values=[1], dense_shape=[1,dim]),
                    tf.SparseTensor(indices=[(tf.sparse.slice(self.adjacency_matrix, [i,0], [1,dim]).indices)[2]], values=[1], dense_shape=[1,dim]),
                    tf.sparse.slice(self.adjacency_matrix, [i+1,0], [dim-i-1,dim])
                ]
            )
            self.adjacency_matrix = tf.sparse.concat(1,
                [
                    tf.sparse.slice(self.adjacency_matrix, [0,0], [dim+2,i]),
                    tf.SparseTensor(indices=[(tf.sparse.slice(self.adjacency_matrix, [0,i], [dim+2,1]).indices)[0]], values=[1], dense_shape=[dim+2,1]),
                    tf.SparseTensor(indices=[(tf.sparse.slice(self.adjacency_matrix, [0,i], [dim+2,1]).indices)[1]], values=[1], dense_shape=[dim+2,1]),
                    tf.SparseTensor(indices=[(tf.sparse.slice(self.adjacency_matrix, [0,i], [dim+2,1]).indices)[2]], values=[1], dense_shape=[dim+2,1]),
                    tf.sparse.slice(self.adjacency_matrix, [0,i+1], [dim+2,dim-i-1])
                ]
            )

            # adding the junction submatrix
            self.adjacency_matrix = tf.sparse.add(self.adjacency_matrix, tf.SparseTensor(indices=[[0+i,1+i],[0+i,2+i],[1+i,0+i],[1+i,2+i],[2+i,0+i],[2+i,1+i]], values=[1,1,1,1,1,1], dense_shape=[dim+2, dim+2]))
        
        # updates the state vector
        self.state_vector = tf.convert_to_tensor(np.array([new_state_vector]).T)

In [364]:
test = Graph([[0,1,1,1],[1,0,1,1],[1,1,0,1],[1,1,1,0]],[[1],[0],[0],[1]])

print("Adjacency Matrix\n", tf.sparse.to_dense(test.adjacency_matrix).numpy())
print("State Vector\n", test.state_vector.numpy())

test.evolve(1279)

print("Adjacency Matrix\n", tf.sparse.to_dense(test.adjacency_matrix).numpy())
print("State Vector\n", test.state_vector.numpy())

Adjacency Matrix
 [[0 1 1 1]
 [1 0 1 1]
 [1 1 0 1]
 [1 1 1 0]]
State Vector
 [[1]
 [0]
 [0]
 [1]]
Adjacency Matrix
 [[0 1 0 0 1 0 0 1]
 [1 0 1 1 0 0 0 0]
 [0 1 0 1 0 1 0 0]
 [0 1 1 0 0 0 0 1]
 [1 0 0 0 0 1 1 0]
 [0 0 1 0 1 0 1 0]
 [0 0 0 0 1 1 0 1]
 [1 0 0 1 0 0 1 0]]
State Vector
 [[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
